# **Clasificación, resumen y extracción de la información**

# **Prácticas 2 y 3**

En las sesiones de prácticas segunda y tercera vamos a programar utilidades básicas para la recuperación de la información y los primeros mecanismos de vectorización de textos. En particular, programaremos índices inversos y las métricas tf-idf.


En este cuaderno Colab, haremos uso de programas que hayáis diseñado en la Práctica 1. La forma más directa de hacer este reuso es que copiéis el código del otro cuaderno y lo peguéis en este.

Para los ejemplos, vamos a trabajar con la igualdad literal de cadenas:

In [ ]:
def coincide (cadena1, cadena2):
  return(cadena1 == cadena2)

Dejaremos para los ejercicios opcionales que exploréis las posibilidades que ofrecen otras definiciones de "coincide" más sofisticadas (con *lematización*, o sinonimias, etc.)

Conviene mantener cargada la posibilidad de *tokenizar*:

In [ ]:
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Definimos el siguiente texto (extraído de un diario digital, sin ningún criterio especial) para ir utilizándolo en los ejemplos:

In [ ]:
texto = "El número de turistas que han visitado España en julio de este año han superado en un 3,1% las cifras pre pandemia. Con su llegada aumentan también los precios del alquiler y la masificación de entornos urbanos y naturales."

In [ ]:
word_tokenize(texto)

['El',
 'número',
 'de',
 'turistas',
 'que',
 'han',
 'visitado',
 'España',
 'en',
 'julio',
 'de',
 'este',
 'año',
 'han',
 'superado',
 'en',
 'un',
 '3,1',
 '%',
 'las',
 'cifras',
 'pre',
 'pandemia',
 '.',
 'Con',
 'su',
 'llegada',
 'aumentan',
 'también',
 'los',
 'precios',
 'del',
 'alquiler',
 'y',
 'la',
 'masificación',
 'de',
 'entornos',
 'urbanos',
 'y',
 'naturales',
 '.']

Nuestro objetivo ahora es, dada una colección textos/documentos, organizados como una lista de cadenas
(y utilizando como identificador su posición en la lista, comenzando en cero), recuperar en qué documentos aparece un término. Para ello, necesitaremos la función "apareceEnTexto" de la práctica anterior (y todas en las que esta se apoye, claro).

Lo hacemos primero sin utilizar ninguna estructura de datos auxiliar.

**Ejercicio 1**. Programar una función que, dada una cadena y una colección de textos estructurada como una lista de cadenas, devuelva una lista con los (índices en la lista de entrada de los) textos en los que aparece la cadena que es el primer argumento.

In [ ]:
def aparece (cadena, listaCadenas):
    for palabra in listaCadenas:
        if coincide(cadena,palabra):
            return True
    return False

In [ ]:
def apareceEnTexto(cadena,texto):
    return aparece(cadena, word_tokenize(texto))

In [ ]:
def localizaEnTextos(cadena, textos):
    indices = []
    i = 0
    for texto in textos:
        if apareceEnTexto(cadena, texto):
            indices.append(i)
        i = i + 1
    return indices

**A la variable *indices* se le ha asignado una lista vacía para almacenar en ella los índices de los textos en los que una cadena determinada aparece. Después, se inicia el contador a cero y se recorre con un bucle, *for texto in textos*, cada uno de los documentos para corroborar si aparece o no el token solicitado en la entrada, mediante la llamada a la función *apareceEnTexto*, previamente ejecutada (antes se ha ejecutado la función *aparece*, pues *aparecEnTexto* se basa en ella). Si la cadena aparece, se guarda el índice determinado en la lista, en tanto que el contador sigue aumentando en cada iteración. De este modo, la función devuelve la lista de los índices textuales, que contiene las posiciones de los textos donde se ha encontrado la cadena.**


In [ ]:
localizaEnTextos('julio',[texto])

[0]

In [ ]:
localizaEnTextos('mayo',[texto])

[]

In [ ]:
localizaEnTextos('turistas',[texto])

[0]

**Los ejemplos corroboran que *julio* y *turistas* aparece en el texto analizado, el primero y único, y que *mayo* no lo hace**.

Definimos ahora otros textos, para poder constituir con ellos una lista de textos (un repositorio o colección o corpus o *dataset*).

In [ ]:
texto1 = "El calentamiento global multiplica por cuatro la intensidad de las lluvias torrenciales en España"

In [ ]:
texto2 = "Ciudadano Musk: el más rico del mundo culmina la transformación de Twitter en su aparato de influencia global"

In [ ]:
texto3 = "La justicia francesa avala la retirada de ayudas públicas a un medio por difundir bulos sobre la salud"

In [ ]:
texto4 = "Estafas, publicidad y miedo a la interacción: por qué cada vez se contesta menos a números que no conocemos"

In [ ]:
textos=[texto,texto1,texto2,texto3,texto4]

In [ ]:
textos

['El número de turistas que han visitado España en julio de este año han superado en un 3,1% las cifras pre pandemia. Con su llegada aumentan también los precios del alquiler y la masificación de entornos urbanos y naturales.',
 'El calentamiento global multiplica por cuatro la intensidad de las lluvias torrenciales en España',
 'Ciudadano Musk: el más rico del mundo culmina la transformación de Twitter en su aparato de influencia global',
 'La justicia francesa avala la retirada de ayudas públicas a un medio por difundir bulos sobre la salud',
 'Estafas, publicidad y miedo a la interacción: por qué cada vez se contesta menos a números que no conocemos']

In [ ]:
localizaEnTextos('la',textos)

[0, 1, 2, 3, 4]

In [ ]:
localizaEnTextos('de',textos)

[0, 1, 2, 3]

In [ ]:
localizaEnTextos('no',textos)

[4]

In [ ]:
localizaEnTextos('en',textos)

[0, 1, 2]

In [ ]:
localizaEnTextos('sol',textos)

[]

El problema de hacerlo así es que cada vez que preguntemos por un término tenemos que recorrer toda la colección de textos. Una forma más eficaz es calcular un *índice inverso*: un diccionario que, dado un término, indica los documentos/textos en los que ese término aparece. Ese diccionario/tabla se denomina "índice inverso" y aunque es costoso en tiempo construirlo, luego la consulta es muy eficaz.

Para comenzar a construir el índice inverso, fijamos un vocabulario de partida, que podría ser calculado (esto puede ser gestionado de modo dinámico: cuando llegue una consulta por un término "nuevo" se podría añadir al diccionario, de forma que el vocabulario estará constituido por las claves del diccionario).

Para tener un elemento de comparación cuando vayamos obteniendo resultados, vamos a fijar el siguiente vocabulario (fijaos en la presencia de *stop words*, que en la mayoría de las aplicaciones prácticas no serían parte de ningún vocabulario).

In [ ]:
vocabulario = ["la", "no", "en", "de", "alquiler","salud", "calentamiento", "mundo"]

**Ejercicio 2**. Programar una función que, dado un vocabulario y una colección de textos estructurada como una lista de cadenas, devuelva un diccionario con claves las palabras del vocabulario y con valor la tupla de índices de los textos en los que aparece esa palabra (utilizad "localizaEnTextos").

In [ ]:
def indiceInverso(vocabulario, textos):
    indice = dict()
    for cadena in vocabulario:
        indice[cadena] = tuple(localizaEnTextos(cadena, textos))
    return indice

In [ ]:
ii = indiceInverso(vocabulario,textos)

In [ ]:
ii

{'la': (0, 1, 2, 3, 4),
 'no': (4,),
 'en': (0, 1, 2),
 'de': (0, 1, 2, 3),
 'alquiler': (0,),
 'salud': (3,),
 'calentamiento': (1,),
 'mundo': (2,)}

**Se ha definido una función que, dado un vocabulario y una colección de textos estructurada como una lista de cadenas, devuelve un diccionario llamado *indice*. Para ello, a la variable *indice* se le ha asignado un diccionario vacío (dict()). Luego, se ha creado un bucle *for* que recorre cada palabra en el vocabulario. En ese bucle a cada palabra se le asigna una clave en el diccionario (indice[palabra]), que es la propia palabra y cuyo valor es una tupla de índices (con lo que los valores, en el diccionario se transforman en inmutables, pues ya no están listados), que indica en qué textos aparece esa palabra. De este modo, se accede a los textos en los que aparece una palabra del vocabulario, sin tener que recorrer la colección de textos en cada búsqueda**.

Una vez que tenemos construido un índice inverso, la extracción de información (es decir, determinar en qué textos aparece un término) es directa y muy sencilla.

**Ejercicio 3**. Programar una función que, dada una cadena y un índice inverso (un diccionario), devuelva una lista con los índices de los textos en los que aparece la cadena que es el primer argumento.

In [ ]:
def localizaEnIndiceInverso(cadena,ii):
    return list(ii[cadena])

In [ ]:
localizaEnIndiceInverso("calentamiento",ii)

[1]

In [ ]:
localizaEnIndiceInverso("no",ii)

[4]

In [ ]:
localizaEnIndiceInverso("la",ii)

[0, 1, 2, 3, 4]

**La función *localizaEnIndiceInverso* devuelve una lista (se ha convertido la tupla en lista para poder cambiar esos elementos) con los índices, es decir, con los valores, de la cadena determinada que es la clave. De esta manera estamos procesando la información para manipularla después con mayor facilidad.**

Descomentad la siguiente celda y ejecutadla.

In [ ]:
localizaEnIndiceInverso("mayo",ii)

KeyError: 'mayo'

**El error ocurre porque *mayo* no se encuentra en el diccionario (índice inverso), por lo que al intentar acceder a una clave que no existe, Python muestra *KeyError*.**

**Ejercicio opcional 1**. Si no habéis tenido precaución, al invocar la función anterior con una cadena que no esté en el vocabulario (o que no esté entre las claves del índice inverso; ¿por qué podría no ser lo mismo?), situación perfectamente razonable, por otra parte, se producirá un error. Aquí se pide programarla para evitar ese error de tres modos diferentes (uno de ellos, o algún otro alternativo que se os ocurra, deberá ser incorporado a vuestra definición, pues no es admisible programar una función que produzca un error ante entradas plausibles).

1. Utilizando el atributo .keys() del diccionario.

2. Utilizando la forma de acceso .get al diccionario.

3. Utilizando excepciones (este es el modo menos elegante, pero las excepciones son un recurso muy útil de los lenguajes de programación que conviene conocer).

**Con respecto a ...*con una cadena que no esté en el vocabulario (o que no esté entre las claves del índice inverso; ¿por qué podría no ser lo mismo?)*, podría no ser lo mismo porque una cadena puede estar en el vocabulario, pero no aparecer en ningún texto, por lo que no tendrá un índice asociado en el diccionario.**

In [ ]:
def localizaEnIndiceInverso(cadena, ii):
    if cadena in ii.keys():
        return list(ii[cadena])
    return []

In [ ]:
def localizaEnIndiceInverso(cadena, ii):
    return list(ii.get(cadena, []))

In [ ]:
def localizaEnIndiceInverso_excepcion(cadena, ii):
    try:
        return list(ii[cadena])
    except KeyError:
        return []


In [ ]:
localizaEnIndiceInverso("buitre",ii)

[]

**Con estas nuevas definiciones se verifica si la clave está en el diccionario y, en caso contrario se devuelve una lista vacía. Así se garantiza la ejecución de la función sin errores.**.

**Ejercicio opcional 2 (abierto)**. Diseñar cómo podría programarse un sistema de búsquedas booleanas a partir de un índice inverso.

Como vocabulario: 'cosas' e 'ideas'.

Como repositorio/corpus/dataset:

textos[0]='cosas veredes'

textos[1]='los hechos son ideas y las ideas son cosas'

textos[2]='ideas son amores'

Ejemplos de posibles ejecuciones:

selecciona("(not 'ideas')",ii)

—> [1]

selecciona("('ideas' and 'cosas')",ii)

—> [2]

selecciona("(('cosas' and (not 'ideas')) or ((not 'cosas') and 'ideas'))",ii)

—> [1,3]


Los índices inversos sirven para hacer búsquedas "exactas" ("booleanas", desde el punto de vista de la lógica); sin embargo en un contexto de *big data*, puede ser interesante recuperar solo los documentos más significativos, por medio de una prelación (*ranked retrieval*). Para abordar este problema, vamos a hacer una pequeña digresión sobre cómo representar los textos.

Si solo estamos interesados en un vocabulario fijado (que puede ser tan amplio como queramos), podemos quedarnos solamente, de cada texto/documento, con la información de qué palabras del vocabulario pertenecen a él. Esa información, aunque es mucho más pobre que el texto en sí, es suficiente, por ejemplo, para establecer el índice inverso. Esa representación de cada frase se denomina SoW (Set of Words).

 # **Ejercicio opcional 2 (abierto)**.

## **Diseño de un sistema de búsqueda booleana con índice inverso**

**Este ejercicio consiste en diseñar un sistema de búsqueda booleana a partir de un índice inverso.**

**El objetivo es procesar consultas booleanas que incluyan los operadores lógicos *AND*, *OR* y *NOT* sobre un corpus de documentos y determinar en cuáles aparece cada término o su combinación.**

**Las consultas se organizan jerárquicamente, de modo que una expresión de primer nivel se corresponde con una consulta completa introducida por el usuario, una de segundo nivel con otra contenida en su interior y así recursivamente.**

**Esta estructura recursiva posibilita la resolución de las consultas de forma progresiva: primero se procesan las expresiones del menor nivel jerárquico y, luego, se combinan sus resultados hasta alcanzar el primer nivel.**

**Para desarrollar este sistema, el problema se ha dividido en varias fases:**

**1. Validación de expresiones (Paso 2): se ha definido la función `expresionValida`, que verifica si una expresión en lenguaje interno está bien formada y contiene únicamente términos presentes en el vocabulario.**

**2. Evaluación de la consulta (Paso 3): una vez validada la estructura, se ha desarrollado la función que evalúa el resultado de la consulta sobre el índice inverso.**

**3. Conversión de lenguaje externo a interno (Paso 1): esta conversión no se ha implementado debido a que implica técnicas de análisis sintáctico *(parsing*) que se desconocen.**


**El enfoque modular adoptado en la resolución del ejercicio favorece la creación y posible ampliación de cada parte independientemente**.

### **a) Definición del corpus y del vocabulario**

**En primer lugar, se ha definido el corpus, la lista de textos que queremos indexar y consultar (en nuestro caso, tres cadenas de texto), así como el vocabulario, que contiene las palabras que formarán las claves del índice inverso: *cosas e ideas*.**


In [ ]:
textos1 = [
    'cosas veredes',
    'los hechos son ideas y las ideas son cosas',
    'ideas son amores'
]

vocabulario = ['cosas', 'ideas']
ii = indiceInverso(vocabulario, textos1)


### **b) Validación recursiva de expresiones**

**Para la validación de las expresiones, se han definido las funciones que verifican si una declaración en formato interno (listas anidadas) está bien constituida y todos sus términos pertenecen al vocabulario (están presentes en el índice inverso)**.

**Las funciones son estructuralmente idénticas, de manera que, cada una reconoce recursivamente un tipo específico de expresión y comprueba que sea una lista, que su longitud sea la esperada y que sus elementos estén correctamente formados (mediante índices como `expresion[0]`, `expresion[1]`, etc.. ):**


**-`es_termino_encapsulado` valida que la expresión sea una lista de un solo elemento, y que ese elemento exista en el índice inverso (es decir, sea un término válido del vocabulario).**
  
**-`es_not_encapsulado` verifica que la expresión sea una lista de dos elementos, en la que el primero sea la cadena *not* (`expresion[0]`) y el segundo otra expresión permitida (`expresion[1]`), validada recursivamente.**

**-`es_and_or_encapsulado` valida listas de tres elementos, en las que los operadores lógicos (*and* u *or*) se encuentran en la posición central (`expresion[1]`) y dos expresiones adecuadas, simples o anidadas,(`expresion[0]` y `expresion[2]`) a ambos lados.**

**La función principal `expresionValida_encapsulado` combina estas tres verificaciones y devuelve *True*, si la expresión coincide con alguna de ellas y *False*, en caso contrario.**

**La validación recursiva se aplica solo en caso necesario, es decir, cuando la expresión contiene formas de niveles inferiores que deben ser validadas independientemente.**


In [ ]:
def es_termino_encapsulado(expresion, indice_inverso):
    return type(expresion) == list and len(expresion) == 1 and expresion[0] in indice_inverso

def es_not_encapsulado(expresion, indice_inverso):
    return (
        type(expresion) == list and
        len(expresion) == 2 and
        expresion[0] == "not" and
        expresionValida_encapsulado(expresion[1], indice_inverso)
    )

def es_and_or_encapsulado(expresion, indice_inverso):
    return (
        type(expresion) == list and
        len(expresion) == 3 and
        expresion[1] in ("and", "or") and
        expresionValida_encapsulado(expresion[0], indice_inverso) and
        expresionValida_encapsulado(expresion[2], indice_inverso)
    )

def expresionValida_encapsulado(expresion, indice_inverso):
    return (
        es_termino_encapsulado(expresion, indice_inverso) or
        es_not_encapsulado(expresion, indice_inverso) or
        es_and_or_encapsulado(expresion, indice_inverso)
    )


#### **b) 1. Ejemplos de validación de expresiones**

**A continuación se muestran algunas llamadas a la función `expresionValida_encapsulado`, que comprueban si ciertas expresiones en formato interno son válidas, de acuerdo con la estructura definida y el vocabulario presente en el índice inverso.**


In [ ]:
expresionValida_encapsulado(["not", ["ideas"]], ii)

True

In [ ]:
expresionValida_encapsulado([["ideas"], "and", ["cosas"]], ii)

True

In [ ]:
expresionValida_encapsulado(
    [
        [["cosas"], "and", ["not", ["ideas"]]],
        "or",
        [["not", ["cosas"]], "and", ["ideas"]]
    ],
    ii)

True

In [ ]:
expresionValida_encapsulado([["patatas"], "and", ["cosas"]], ii)

False

### **c) Evaluación de la consulta**

**Una vez validada la estructura lógica de las expresiones, el siguiente paso es su evaluación (fase 3 del sistema, se asume que la expresión es sintácticamente correcta y contiene únicamente términos válidos presentes en el índice inverso) con el objetivo de determinar en qué documentos del corpus se cumple la condición indicada por la expresión booleana.**

**Dado que las expresiones pueden estar compuestas por operadores lógicos *(and, or, not)* y contener otras expresiones anidadas, la evaluación también se resuelve recursivamente. De este modo, la función `evalua_expresion` recibe dos argumentos:**

**`expr`,la expresión booleana en formato interno (listas anidadas)
e `indice_inverso`, el diccionario que asocia cada término con los índices de los textos donde aparece.**

**Si la expresión es un término válido del vocabulario, se accede directamente a la lista de documentos en que aparece en el índice inverso:**

**if es_termino_encapsulado(expr, indice_inverso):
    return list(indice_inverso[expr[0]])**

**Por otro lado, `es_not_encapsulado(expr, indice_inverso)`comprueba si la expresión representa una negación. Si se cumple la condición, `range(len(textos1))` se generan los índices correspondientes a todos los textos del corpus para, a partir de este conjunto, eliminarse los que corresponden a los textos en los que se cumple la subexpresión negada. Asimismo, la función `evalua_expresion(expr[1], indice_inverso)` se encarga de evaluar la subexpresión que aparece anidada en el operador `"not"`. A partir del resultado, una comprensión de listas recorre los índices de los textos del corpus y selecciona los que no aparecen en la evaluación interna.**

**Por último, `es_and_or_encapsulado(expr, indice_inverso) `evalúa expresiones que combinan los operadores lógicos *and* u *or* de la siguiente manera: se evalúan por separado los dos lados de la expresión: el izquierdo (expr[0]) y el derecho (expr[2]), y se devuelven dos listas de índices con los textos que cumplen cada parte. Si el operador es *and*, la salida es una lista con los índices que aparecen en ambas listas, si es *or*, se unen ambas, hecho que da como resultado las referencias a los textos que cumplen, al menos, una de las dos condiciones.**


In [ ]:
def evalua_expresion(expr, indice_inverso):
    if es_termino_encapsulado(expr, indice_inverso):
        return list(indice_inverso[expr[0]])

    if es_not_encapsulado(expr, indice_inverso):
        return [i for i in range(len(textos1)) if i not in evalua_expresion(expr[1], indice_inverso)]

    if es_and_or_encapsulado(expr, indice_inverso):
        izq = evalua_expresion(expr[0], indice_inverso)
        der = evalua_expresion(expr[2], indice_inverso)
        if expr[1] == "and":
            return [i for i in izq if i in der]
        elif expr[1] == "or":
            resultado = list(izq)
            for i in der:
                if i not in resultado:
                    resultado.append(i)
            return resultado

    return []


#### **c) 1. Ejemplos de evaluación de consultas**

In [ ]:
evalua_expresion(["not", ["ideas"]], ii)

[0]

In [ ]:
(evalua_expresion([["ideas"], "and", ["cosas"]], ii))

[1]

In [ ]:
(evalua_expresion(
    [
        [["cosas"], "and", ["not", ["ideas"]]],
        "or",
        [["not", ["cosas"]], "and", ["ideas"]]
    ],
    ii))

[0, 2]

In [ ]:
evalua_expresion([["patatas"], "and", ["cosas"]], ii)

[]

**Ejercicio 4**. Programar una función que, dado un texto (una cadena de caracteres) y un vocabulario (una lista de cadenas, sin repeticiones), devuelva la lista de palabras del vocabulario que aparecen en el texto (utilizad "apareceEnTexto"). ¿Lo podríais hacer en una única línea usando compresión de listas?

In [ ]:
def apareceEnTexto(cadena,texto):
    return aparece(cadena, word_tokenize(texto))

In [ ]:
def sow (texto, vocabulario):
    return [cadena for cadena in vocabulario if apareceEnTexto(cadena, texto)]

In [ ]:
sow(texto,vocabulario)

['la', 'en', 'de', 'alquiler']

In [ ]:
sow(texto1,vocabulario)

['la', 'en', 'de', 'calentamiento']

In [ ]:
sow(texto2,vocabulario)

['la', 'en', 'de', 'mundo']

In [ ]:
sow(texto3,vocabulario)

['la', 'de', 'salud']

In [ ]:
sow(texto4,vocabulario)

['la', 'no']

**La función *sow* se apoya en la función apareceEnTexto para, dado un texto y un vocabulario, determinar, mediante la iteración palabra a palabra, si una cadena determinada del vocabulario aparece en el texto. Con la comprensión de listas se devuelve una lista con todas las palabras del vocabulario que han sido detectadas en el texto.
En los ejemplos se observa que el término *la* del vocabulario aparece en todos los documentos, pues es una *stop words*, y que si quiero consultar un documento sobre el calentamiento global, por ejemplo, buscaré en el documento 2.**

Cada sow admite una representación como conjunto (como en el ejemplo anterior) o bien como un vector de dimensión la cardinalidad del vocabulario (representación densa) o como un diccionario (representación dispersa; *sparse* en inglés).

**Ejercicio 5**. Programar una función que, dado un texto (una cadena de caracteres) y un vocabulario (una lista de cadenas, sin repeticiones), devuelva un vector de 0s y 1s, con un 1 en la posición en la que un término del vocabulario aparezca en el texto (utilizad "apareceEnTexto").

In [ ]:
def sow_vector(texto, vocabulario):
    vector = []
    for cadena in vocabulario:
        if apareceEnTexto(cadena, texto):
            vector.append(1)
        else:
            vector.append(0)
    return vector

In [ ]:
sow_vector(texto,vocabulario)

[1, 0, 1, 1, 1, 0, 0, 0]

**La función sow_vector toma como entrada un texto y un vocabulario y devuelve un vector que indica la presencia o ausencia de cada palabra del vocabulario en el texto.
Para definirla, se crea un vector como una lista vacía que almacenará los valores 0 o 1. Posteriormente, se utiliza la función apareceEnTexto para comprobar si cada palabra presente en el vocabulario aparece en el texto; si es así, se añade un 1 al vector con *append*, y si no aparece, se añade un 0. Finalmente, la función devuelve el vector con los valores correspondientes.**



Se lee mejor en paralelo (intentad entender el código):

In [ ]:
def muestra_sow_vector(texto,vocabulario):
  print("La representación de: \n" + texto + "\nes la siguente:")
  for cadena,valor in zip(vocabulario,sow_vector(texto, vocabulario)):
    print(cadena + " : " + str(valor))

In [ ]:
muestra_sow_vector(texto,vocabulario)

La representación de: 
El número de turistas que han visitado España en julio de este año han superado en un 3,1% las cifras pre pandemia. Con su llegada aumentan también los precios del alquiler y la masificación de entornos urbanos y naturales.
es la siguente:
la : 1
no : 0
en : 1
de : 1
alquiler : 1
salud : 0
calentamiento : 0
mundo : 0


In [ ]:
muestra_sow_vector(texto1,vocabulario)

La representación de: 
El calentamiento global multiplica por cuatro la intensidad de las lluvias torrenciales en España
es la siguente:
la : 1
no : 0
en : 1
de : 1
alquiler : 0
salud : 0
calentamiento : 1
mundo : 0


**Entiendo que la función *muestra_sow_vector* imprime el texto y su representación vectorial como sow, consistente en la de la palabra del vocabulario que aparece en el  texto y su correspondiente valor binario (1 si aparece, 0 si no aparece), a partir de la función *zip* que "descomprime" el vector generado por la función sow-vector, al asignar a cada valor su clave.
De este modo, se facilita la interpretación numérica y se observan rápidamente las palabras presentes y ausentes en el texto**

**Ejercicio 6**. Programar una función que, dado un texto (una cadena de caracteres) y un vocabulario (una lista de cadenas, sin repeticiones), devuelva un diccionario en el que las claves sean los términos del vocabulario que aparezcan en el texto (utilizad "apareceEnTexto").

In [ ]:
def sow_dic (texto, vocabulario):
    dict={}
    for cadena in vocabulario:
      if apareceEnTexto(cadena,texto):
        dict[cadena]=1
    return dict

**La función comienza creando un diccionario vacío llamado dict. Después, recorre cada término del vocabulario utilizando un bucle *for*. Para cada palabra del vocabulario, mediante la llamada a la función *apareceEnTexto*, que verifica si la palabra está en él. Si la palabra aparece, se añade al diccionario con un valor de 1.

Finalmente, la función devuelve el diccionario con las claves correspondientes a las palabras que aparecen en el texto y con el valor de 1 asignado a cada una de ellas.**

In [ ]:
sow_dic(texto,vocabulario)

{'la': 1, 'en': 1, 'de': 1, 'alquiler': 1}

In [ ]:
sow_dic(texto,vocabulario).keys()

dict_keys(['la', 'en', 'de', 'alquiler'])

**Con esta función se ha devuelto la lista de las claves. Lo que demuestra que la función programada cumple con su propósito.Se puede considerar una versión más compacta que *sow_vector*, ya que, en lugar de representar todas las palabras del vocabulario con ceros y unos, solo almacena las que están presentes en el texto.Si *sow vector* fuera una representación dispersa, podríamos considerarla la función inversa de *sow_dict.*, pues ambas almacenan las palabras presentes en el vocabulario, pero mediante el valor y la clave, respectivamente.**

In [ ]:
vocabulario

['la', 'no', 'en', 'de', 'alquiler', 'salud', 'calentamiento', 'mundo']

La información contenidas en los sows es suficiente para construir el índice inverso. En primer lugar, vamos detectar en que sows (implementados como diccionarios) aparece un término.

**Ejercicio 6**. Programar una función que, dada una cadena y una lista de sows (como diccionarios), devuelva una lista con los índices de los sows en lo que aparezca la cadena de entrada.

In [ ]:
def localizaEnSows(cadena, sows):
    indices = []
    contador = 0
    for sow in sows:
        if cadena in sow:
            indices.append(contador)
        contador+=1
    return indices

In [ ]:
localizaEnSows('de',[sow_dic(texto,vocabulario) for texto in textos])

[0, 1, 2, 3]

**Esta función, a partir de una cadena y una lista de *sows*, devuelve una lista con los índices de los *sows* en los que aparece la cadena de entrada. Parte de la creación de una lista vacía (indices) y de un contador, iniciado en cero. A través de un bucle, se itera por cada diccionario en la lista de *sows*. Si la palabra aparece en el *sow*, es decir, si está entre las claves del diccionario que representa el texto, entonces se ejecuta *indices.append(contador)*, una función que añade el *contador* correpondiente a la lista *indices*. Este proceso se repite en cada iteración, en la que el contador se incrementa en 1 para reflejar la posición del siguiente *sow*. Finalmente, se devuelve la lista con los índices correspondientes.**


**Ejercicio 7**. Programar una función que, dado un vocabulario y una lista de sows (como diccionarios), devuelva el índice inverso de la colección de textos (representada por la lista de sows) del vocabulario (utilizando "localizaEnSows").

In [ ]:
def iiFromSows(vocabulario,sows):
    ii = dict()
    for cadena in vocabulario:
        ii[cadena] = localizaEnSows(cadena, sows)
    return ii

**Esta función usa las representaciones vectoriales para encontrar en qué textos aparece cada término del vocabulario, en lugar de recorrer los textos completos como en la función *indiceInverso*.
Para ello, primero se crea un diccionario vacío llamado *ii*, que será el índice
inverso. A continuación, se itera sobre cada palabra en la lista de vocabulario, *for cadena in vocabulario:*, y cada palabra del vocabulario, mediante la lista con los índices de los sows en los que aparece, se almacena como valor de la clave correspondiente en el diccionario ii, que se asigna mediante la llamada a la a la función localizaEnSows(cadena, sows), para que la función pueda devolver el índice inverso, en el que cada palabra del vocabulario tiene como valor la lista de índices de los textos en los que está presente**.

In [ ]:
iiFromSows(vocabulario,[sow_dic(texto,vocabulario) for texto in textos])

{'la': [0, 1, 2, 3, 4],
 'no': [4],
 'en': [0, 1, 2],
 'de': [0, 1, 2, 3],
 'alquiler': [0],
 'salud': [3],
 'calentamiento': [1],
 'mundo': [2]}

**Esta celda muestra una llamada a la función *iiFromSows* con dos argumentos: a)las palabras del vocabulario que queremos analizar y b) una comprensión de listas, que recorre cada texto y genera un diccionario en el que las claves son los términos del vocabulario que aparezcen en el texto (sow_dic).
Dado que *iiFromSows* se basa en *localizaEnSows* la salida muestra, en forma de diccionario, no solo las claves, sino también sus correspondientes valores, las listas de índices de los textos en los que aparecen.
Se ejemplifica así como las funciones se van componiendo y encadenan las transformaciones acumuladas en sus definicones.**



Al pasar de un texto a un sow hemos perdido mucha información (aunque hemos mantenido la suficiente como para construir el índice inverso) de tipo gramatical, pero también otra que puede ser interesante si queremos hacer una extracción de la información con prelación: el número de veces que una palabra aparece en un texto. Almacenar esa información es lo que se conoce como pasar a un modo de representación BoW: *Bag of Words*. La representación puede ser tanto densa como dispersa. Programamos solo al segunda.

**Ejercicio 8**. Programar una función que, dado un texto y un vocabulario, devuelva un diccionario que represente el bow del texto respecto al vocabulario (utilícese la función "frecuencia" de la Práctica 1).

In [ ]:
def frecuencia(cadena, listaCadenas):
    contador = 0
    for palabra in listaCadenas:
        if coincide(cadena, palabra):
            contador= contador + 1
    return contador

In [ ]:
def bow_dic (texto, vocabulario):
    bow_dict = {}
    for cadena in vocabulario:
        bow_dict[cadena] = frecuencia(cadena, word_tokenize(texto))
    return bow_dict

**La función *bow_dic* tiene dos argumentos: un texto y un vocabulario (lista de palabras sin repeticiones). En primer lugar, a la variable *bow* se le asigna un diccionario, a continuación, con un bucle *for*, se itera por cada palabra del vocabulario, de modo que, mediante la llamada a la función *frecuencia*,  el valor obtenido se guarda como el correspondiente a la clave (la cadena) en ese diccionario. Se devuelve así el diccionario que representa el bow (*Bag of Words*) bajo la forma de esa estructura.**

In [ ]:
bow_dic(texto,vocabulario)

{'la': 1,
 'no': 0,
 'en': 2,
 'de': 3,
 'alquiler': 1,
 'salud': 0,
 'calentamiento': 0,
 'mundo': 0}

In [ ]:
bow_dic(texto1,vocabulario)

{'la': 1,
 'no': 0,
 'en': 1,
 'de': 1,
 'alquiler': 0,
 'salud': 0,
 'calentamiento': 1,
 'mundo': 0}

**Los ejemplos muestran que la función bow_dict se ejecuta correctamente y que permite identificar de un modo rápido las similitudes y diferencias temáticas entre los textos. Además, evidencian que, según el propósito puede ser conveniente eliminar las *stopwords*, por ejemplo, en tareas como la clasificación de textos o el análisis de sentimientos, su eliminación es positiva; sin embargo, en otros estudios lingüísticos mantenerlas puede ser útil para comprender mejor la composición textual. No obstante, el modelo bow tiene una limitación importante: representa los textos de manera absoluta, sin considerar el contexto del repositorio en el que se encuentran. Esto puede hacer que términos muy frecuentes en todos los documentos pierdan capacidad discriminativa. Para resolver este problema, se emplea la métrica TF-IDF (Term Frequency - Inverse Document Frequency), que pondera la frecuencia de los términos según su importancia en la colección de textos.**

Ahora la idea es, dado un texto como una lista de bows, organizar el índice inverso ordenando los documentos en los que aparece un términos de mayor a menor frecuencia según indica cada bow.

Sin embargo, esto no tiene en cuenta la importancia del término dentro de toda la colección de documentos. Es decir, queremos dar más peso a aquellos términos dentro de un documento que sean frecuentes en él, pero no aparezcan demasiado en el resto de documentos. Para eso se introduce la medida tf-idf de cada término en cada documento de una colección de documentos.

**Ejercicio 9**. Programar una función que, dada una cadena y un lista de bows, devuelva el número de documentos de la lista en la que la cadena aparece.

In [ ]:
def df(cadena, bows):
    contador = 0
    for bow in bows:
        if cadena in bow and bow[cadena] > 0:
            contador += 1
    return contador

**La función *df* tiene dos argumentos: cadena y bows (lista de diccionarios, donde cada bow representa un texto y almacena las frecuencias de las palabras en ese texto). Primero se crea la variable *contador*, que se inicia en cero. Luego, se itera sobre cada *bow* en la lista de *bows* y, si la cadena está en el diccionario y su valor es mayor de cero (para que no se computen las que tienen 0 como valor) se suma uno al contador.**

**Finalmente, la función devuelve el número de diccionarios en los que aparece la cadena, es decir, la cantidad de textos en los que la palabra aparece**

In [ ]:
bows = [bow_dic(texto,vocabulario) for texto in textos]

**En esta comprensión de listas, la variable *bows* es una lista de diccionarios, en la que cada diccionario representa un texto en formato *bag of words*. Primero, se itera cada texto, seguidamente se llama a la función *bow_dic(texto, vocabulario)*, que genera un diccionario *bow* para ese texto. Finalmemte, el diccionario generado se agrega a la lista *bows*.
El resultado es una lista de diccionarios, en la que cada diccionario representa un documento en formato *bow*.**

In [ ]:
bows

[{'la': 1,
  'no': 0,
  'en': 2,
  'de': 3,
  'alquiler': 1,
  'salud': 0,
  'calentamiento': 0,
  'mundo': 0},
 {'la': 1,
  'no': 0,
  'en': 1,
  'de': 1,
  'alquiler': 0,
  'salud': 0,
  'calentamiento': 1,
  'mundo': 0},
 {'la': 1,
  'no': 0,
  'en': 1,
  'de': 2,
  'alquiler': 0,
  'salud': 0,
  'calentamiento': 0,
  'mundo': 1},
 {'la': 2,
  'no': 0,
  'en': 0,
  'de': 1,
  'alquiler': 0,
  'salud': 1,
  'calentamiento': 0,
  'mundo': 0},
 {'la': 1,
  'no': 1,
  'en': 0,
  'de': 0,
  'alquiler': 0,
  'salud': 0,
  'calentamiento': 0,
  'mundo': 0}]

In [ ]:
df("la",bows)

5

In [ ]:
df("en",bows)

5

In [ ]:
df("salud",bows)

1

In [ ]:
df("mayo",bows)

0

In [ ]:
t

**La frecuencia de documentos en los que una palabra aparece (DF) es necesaria para calcular el *Inverse Document Frequency* (IDF), requerido, a su vez, en la métrica TF-IDF, empleada para calcular la relevancia de una palabra en un documento con respecto a un conjunto de textos. Dado que, cuando una palabra aparece en muchos documentos (DF alto), su IDF es bajo y a la inversa, en este corpus, palabras como *en* o *la* son irrelevantes para distinguir entre textos porque su DF, 5, es igual al número total de documentos (N), y, debido a que log(1)=0 → TF-IDF = TF * IDF = 0.**





**Ejercicio opcional 3**. Vamos a calcular la frecuencia de un término dentro de una colección (*collection frequency*, cf). De dos formas:

1. Utilizando la representación dispersa de bows, por medio de diccionario, como hemos venido haciendo.

2. Utilizando una representación densa de bows (como vectores de números) y el tratamiento de matrices que planteamos en el Ejercicio opcional 2 de la Práctica 1.

Para definir el valor idf vamos a tener que calcular logaritmos. Necesitamos importar la librería math:

In [ ]:
import math

El logaritmo se utiliza para *suavizar* el crecimiento (o decrecimiento) brusco de los números. Cuando solo nos interesa comparar dos números (para saber cual de ellos es más grande) podemos comparar sus logaritmos, sirviendo para el mismo fin (porque el logaritmo es una función creciente: x < y <==> log(x) < log(y)), y limitando el peligro de "overflow" o "underflow".

In [ ]:
x = 10
for i in range(10):
  print ('Valor: ' + str(x) + "  | su logaritmo: " + str(math.log(x)))
  print ('Valor: ' + str(1/x) + "  | su logaritmo: " + str(math.log(1/x)))
  x = x * 100


Valor: 10  | su logaritmo: 2.302585092994046
Valor: 0.1  | su logaritmo: -2.3025850929940455
Valor: 1000  | su logaritmo: 6.907755278982137
Valor: 0.001  | su logaritmo: -6.907755278982137
Valor: 100000  | su logaritmo: 11.512925464970229
Valor: 1e-05  | su logaritmo: -11.512925464970229
Valor: 10000000  | su logaritmo: 16.11809565095832
Valor: 1e-07  | su logaritmo: -16.11809565095832
Valor: 1000000000  | su logaritmo: 20.72326583694641
Valor: 1e-09  | su logaritmo: -20.72326583694641
Valor: 100000000000  | su logaritmo: 25.328436022934504
Valor: 1e-11  | su logaritmo: -25.328436022934504
Valor: 10000000000000  | su logaritmo: 29.933606208922594
Valor: 1e-13  | su logaritmo: -29.933606208922594
Valor: 1000000000000000  | su logaritmo: 34.538776394910684
Valor: 1e-15  | su logaritmo: -34.538776394910684
Valor: 100000000000000000  | su logaritmo: 39.14394658089878
Valor: 1e-17  | su logaritmo: -39.14394658089878
Valor: 10000000000000000000  | su logaritmo: 43.74911676688687
Valor: 1e-19

**Ejercicio 10**. Programar una función que, dada una cadena y un lista de bows, devuelva el valor idf de la cadena (respecto a alguna de las definiciones explicadas en teoría).

In [ ]:
def idf (cadena, bows):
    N=len(bows)
    document_frequency =df(cadena,bows)
    return math.log((1 + N) / (1 +  document_frequency))

**La función *idf* (Inverse Document Frequency), dada una cadena y una lista de bows, se ha definido para medir la importancia de un término en una colección de documentos. De esta manera, a la variable N se le ha asignado el número de bows calculados por la función de Python *len* y la variable *document_frequency* almacena el resultado de la función *df(cadena, bows)* que calcula el número de documentos en los que la palabra aparece, o sea su frecuencia.**

**Esta función devuelve, mediante la librería *math* el cálculo de la fórmula de IDF= log((1 + N) / (1 + df)), en la que, dado que la división entre cero no está definida para los reales, se añade un uno al numerador y al denominador para conservar la escala y evitar esa indefinicón. Además, debido a que la función logarítmica es la inversa de la función exponencial, con su aplicación se revierte el crecimiento exponencial de la frecuencia documental de ciertos términos, al tiempo que los valores resultantes son más manejables.**


In [ ]:
idf("la",bows)

0.0

In [ ]:
idf("en",bows)

0.4054651081081644

In [ ]:
idf("salud",bows)

1.0986122886681098

In [ ]:
idf("mayo",bows)

1.791759469228055

**Estos ejemplos evidencian la relación inversa entre la frecuencia documental (DF) y el *Inverse Document Frequency (IDF)*, mencionada en el ejercicio anterior. Tokens como *la* que aparece en los cinco documentos del corpus (df = 5), tienen un IDF de 0, lo que confirma que una alta frecuencia terminológica en la colección reduce su capacidad para diferenciar textos.
Otros, como *salud*, que solo aparece en uno tiene el máximo idf, por lo que es una palabra clave.**


Descomentad y ejecutad la siguiente celda.

In [ ]:
idf("mayo",bows)

1.791759469228055

**El hecho de que el IDF de *mayo* sea ≈ 1, 79 se debe a que IDF(mayo) = log( 1+5/1) = log(6) ≈1, 79, es decir, en el caso de *salud* con el máximo DF=5, su IDF es log(1)= 0.
Sin embargo, al evaluar cualquier término que no aparezca en ningún documento en la función logaritmo, el resultado será un número positivo  porque hemos sumado uno al numerador y denominador para evitar que la solución no esté definida en los reales. Es por tanto una cuestión de haber evitado esa división entre cero.
Además, debido a que el TF = 0 en la métrica TF-IDF tenemos: TF*IDF= 0
independientemente de que el IDF sea positivo, por lo que en la búsqueda de información, *mayo* no contribuirá a la similitud entre documentos porque su peso es nulo.**


La representación con el modelo BoW ya calcula la frecuencia del término en un texto (tf, *term frequency*). Esencialmente, hay que devolver bow[cadena]. Pero hay que tener cuidado de que si la cadena por la que se pregunta no aparece en el texto, se devuelva 0:

In [ ]:
def tf (cadena, bow):
  return (bow.get(cadena,0))

**Esta función, que tiene por argumentos una cadena y un bow y que devuelve la frecuencia de un término en un texto, emplea el método *get()*  para tener por salida el valor de cero si el término no está. Así se evita un *Key error* como el observado en un ejercicio anterior.**



In [ ]:
tf("de",bows[0])

3

In [ ]:
tf("mayo",bows[0])

0

In [ ]:
tf("salud",bows[0])

0

In [ ]:
tf("salud",bows[4])

0

**Estos ejemplos muestran la frecuencia de estos términos en el primer texto, Bow[0] y en el quinto. Se verifica, así, que la función está bien definida.**

Ahora para calcular el valor de tf-idf basta multiplicar tf por idf.

**Ejercicio 11**. Programar una función que, dada una cadena, un bow y un lista de bows, devuelva el valor tf-idf de la cadena en el texto bow respecto a la colección de textos bows (una única línea).

In [ ]:
def tfxidf (cadena,bow,bows):
    return tf(cadena, bow) * idf(cadena, bows)


**Esta función necesita tres argumentos porque los tres se relacionan con la métrica TF-IDF: la cadena es el término cuya frecuencia se mide, el diccionario *bag of words* (bow) se relaciona con la TF, la frecuencia de la cadena en un documento, y la lista de bows se vincula con la DF y la IDF (métricas de la frecuencia de esa cadena en todos los documentos). La función devuelve el valor de TF-IDF mediante el producto de esas dos métricas.**

In [ ]:
tfxidf("la",bows[0],bows)

0.0

In [ ]:
tfxidf("de",bows[0],bows)

0.5469646703818638

In [ ]:
tfxidf("en",bows[0],bows)

0.8109302162163288

In [ ]:
tfxidf("salud",bows[0],bows)

0.0

In [ ]:
tfxidf("salud",bows[1],bows)

0.0

In [ ]:
tfxidf("salud",bows[2],bows)

0.0

In [ ]:
bows[2]

{'la': 1,
 'no': 0,
 'en': 1,
 'de': 2,
 'alquiler': 0,
 'salud': 0,
 'calentamiento': 0,
 'mundo': 1}

In [ ]:
tfxidf("mundo",bows[2],bows)

1.0986122886681098

**Los ejemplos muestran cómo la métrica TF-IDF refleja la relevancia de una palabra dentro de un documento, al tiempo que se tiene en cuenta su frecuencia en el conjunto de documentos. Se destacan los siguientes aspectos: 1) TF-IDF es 0 cuando la palabra no aparece en un documento porque, dado que el  cáculo es documento a documento, si no aparece, lógicamente no es relevante y el producto es 0. 2) Palabras muy comunes tienen TF-IDF bajo o nulo, por ejemplo, *la*, que aparece en todos los documentos, tiene un TF-IDF nulo porque, como vimos su IDF=0. Se observa, entonces que TF-IDF asigna mayor peso a términos que aparecen con alta frecuencia en un documento específico, pero con baja frecuencia en el repositorio. En el caso de *mundo*, dado que solo aparece una vez en un único documento y no en los demás, su IDF es alto, por lo que tiene más peso en comparación con palabras como *la*, que aparece en todos los documentos.
En conclusión, esta métrica penaliza a las palabras muy frecuentes por poco significativas.**

**Ejercicio 12**. Programar una función que, dado un bow y un lista de bows, devuelva un diccionario en el que a cada término del bow de entrada se le asocie su valor tf-idf.

In [ ]:
def bow_tfidf(bow, bows):
    tfidf_dict = {}
    for cadena in bow:
        tfidf_dict[cadena] = tfxidf(cadena, bow, bows)
    return tfidf_dict

**Para definir esta función (que calcula el TF-IDF de un único documento), cuyos argumentos son un bow y una lista de bows,se ha creado un diccionario vacío y, mediante un bucle *for*, que recorre cada cadena en un bow, calcula el TF-IDF de cada palabra, teniendo en cuenta los parámetros *cadena, bow y bows*. Una vez calculado lo almacena en el diccionario en el que la cadena es la clave y el valor es el TF-IDF.**

In [ ]:
resultado = bow_tfidf(bows[0], bows)
print(resultado)

{'la': 0.0, 'no': 0.0, 'en': 0.8109302162163288, 'de': 0.5469646703818638, 'alquiler': 1.0986122886681098, 'salud': 0.0, 'calentamiento': 0.0, 'mundo': 0.0}


In [ ]:
resultado = bow_tfidf(bows[1], bows)
print(resultado)

{'la': 0.0, 'no': 0.0, 'en': 0.4054651081081644, 'de': 0.1823215567939546, 'alquiler': 0.0, 'salud': 0.0, 'calentamiento': 1.0986122886681098, 'mundo': 0.0}


In [ ]:
resultado = bow_tfidf(bows[2], bows)
print(resultado)

{'la': 0.0, 'no': 0.0, 'en': 0.4054651081081644, 'de': 0.3646431135879092, 'alquiler': 0.0, 'salud': 0.0, 'calentamiento': 0.0, 'mundo': 1.0986122886681098}


In [ ]:
resultado = bow_tfidf(bows[3], bows)
print(resultado)

{'la': 0.0, 'no': 0.0, 'en': 0.0, 'de': 0.1823215567939546, 'alquiler': 0.0, 'salud': 1.0986122886681098, 'calentamiento': 0.0, 'mundo': 0.0}


In [ ]:
resultado = bow_tfidf(bows[4], bows)
print(resultado)

{'la': 0.0, 'no': 1.0986122886681098, 'en': 0.0, 'de': 0.0, 'alquiler': 0.0, 'salud': 0.0, 'calentamiento': 0.0, 'mundo': 0.0}


**De los ejemplos se desprende que las palabras con un TF-IDF más alto son: *alquiler, calentamiento, mundo, salud y no*. Estos términos tienen un IDF-TF de  1.0986, es decir, son relevantes en el documento en que aparecen y poco frecuentes en el resto del repositorio. Estas voces indican la variabilidad de un documento frente a otro y permitirán clasificarlo con precisión.
Además, se observa que el máximo valor de TF-IDF es  1.0986, porque las palabras clave solo aparecen una vez por documento.**

**Ejercicio 13**. Programar una función que, dada una lista de bows, devuelva una lista de diccionarios cada uno de los cuales corresponde al de valores tf-idf de cada uno de los bows de la lista de entrada.

In [ ]:
def tfxidf_model(bows):
    tfidf_lista = []
    for bow in bows:
        tfidf_lista.append(bow_tfidf(bow, bows))
    return tfidf_lista

**Se ha programado una función llamada *tfxidf_model(bows)* que calcula los valores TF-IDF para todos los documentos representados en la lista bows. De este modo, se ha creado una lista y, a través de un bucle *for* se ha iterado en cada bow de la lista de bows. En cada iteración, calcula el TF-IDF del documento iterado de la siguiente manera: primero, la función *bow_tfidf(bow, bows)* calcula los valores TF-IDF para un bow, teniendo por referencia todos los bows. En segundo lugar, el método *append()* agrega este diccionario a la lista tfidf_lista previamente creada. Finalmente, la función devuelve esta lista, en la que cada documento es un diccionario que contiene los valores TF-IDF.**

In [ ]:
tfxidf_model(bows)

[{'la': 0.0,
  'no': 0.0,
  'en': 0.8109302162163288,
  'de': 0.5469646703818638,
  'alquiler': 1.0986122886681098,
  'salud': 0.0,
  'calentamiento': 0.0,
  'mundo': 0.0},
 {'la': 0.0,
  'no': 0.0,
  'en': 0.4054651081081644,
  'de': 0.1823215567939546,
  'alquiler': 0.0,
  'salud': 0.0,
  'calentamiento': 1.0986122886681098,
  'mundo': 0.0},
 {'la': 0.0,
  'no': 0.0,
  'en': 0.4054651081081644,
  'de': 0.3646431135879092,
  'alquiler': 0.0,
  'salud': 0.0,
  'calentamiento': 0.0,
  'mundo': 1.0986122886681098},
 {'la': 0.0,
  'no': 0.0,
  'en': 0.0,
  'de': 0.1823215567939546,
  'alquiler': 0.0,
  'salud': 1.0986122886681098,
  'calentamiento': 0.0,
  'mundo': 0.0},
 {'la': 0.0,
  'no': 1.0986122886681098,
  'en': 0.0,
  'de': 0.0,
  'alquiler': 0.0,
  'salud': 0.0,
  'calentamiento': 0.0,
  'mundo': 0.0}]

In [ ]:
bows

[{'la': 1,
  'no': 0,
  'en': 2,
  'de': 3,
  'alquiler': 1,
  'salud': 0,
  'calentamiento': 0,
  'mundo': 0},
 {'la': 1,
  'no': 0,
  'en': 1,
  'de': 1,
  'alquiler': 0,
  'salud': 0,
  'calentamiento': 1,
  'mundo': 0},
 {'la': 1,
  'no': 0,
  'en': 1,
  'de': 2,
  'alquiler': 0,
  'salud': 0,
  'calentamiento': 0,
  'mundo': 1},
 {'la': 2,
  'no': 0,
  'en': 0,
  'de': 1,
  'alquiler': 0,
  'salud': 1,
  'calentamiento': 0,
  'mundo': 0},
 {'la': 1,
  'no': 1,
  'en': 0,
  'de': 0,
  'alquiler': 0,
  'salud': 0,
  'calentamiento': 0,
  'mundo': 0}]

**Mientras que en la representación *bows*, los términos tienen por valores solo la cantidad de veces que aparecen en el documento, o sea, su frecuencia
en la llamada a la función *tfxidf_model(bows)*, se observa la transformación a pesos de esos valores, tras haber aplicado la métrica TF-IDF para evaluar su importancia en el corpus.
Asimismo, en los ejemplos se refleja lo ya mencionado: que la métrica TF-IDF penaliza las palabras más frecuentes, como *la* por no ser discrimitarorias documentalmente.**

En las siguientes celdas vamos a generar el modelo tf-idf que proporciona la librería sklearn, y a extraerlo de forma que lo podamos comparar con el que acabamos de calcular.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer(stop_words=None, vocabulary=vocabulario) # para comparar, el mismo vocabulario

In [ ]:
feature_names = vectorizer.get_feature_names_out()

In [ ]:
feature_names

array(['la', 'no', 'en', 'de', 'alquiler', 'salud', 'calentamiento',
       'mundo'], dtype=object)

In [ ]:
tf_idf = vectorizer.fit_transform(textos)

In [ ]:
print(tf_idf)

  (0, 0)	0.19654575253083423
  (0, 2)	0.5524763946578989
  (0, 3)	0.6971408403404858
  (0, 4)	0.41247333154673
  (1, 0)	0.3375338265523302
  (1, 2)	0.4743920160255332
  (1, 3)	0.39907351927997176
  (1, 6)	0.7083526362438907
  (2, 0)	0.27765951098422603
  (2, 2)	0.3902407546227053
  (2, 3)	0.6565656505710366
  (2, 7)	0.5826996618170748
  (3, 0)	0.7797586601687984
  (3, 3)	0.30730849100478064
  (3, 5)	0.5454703688085403
  (4, 0)	0.430165282498796
  (4, 1)	0.9027501480103624


Lo que devuelve "vectorizer.fit_transform" es una matriz dispersa (*sparse*):

In [ ]:
tf_idf[0,4]

0.41247333154673

**Con estos ejemplos se compara el cálculo automático TF-IDF, ejecutado por la herramienta *fidfVectorizer* de la librería *sklearn* con el manual realizado en el cuaderno. Para ello, tras importarse la aplicación *fidfVectorizer*, se vectoriza el vocabulario empleado en la práctica, sin elimar las *stop words* para que la comparación sea equilibrada. Después, se obtiene la lista de palabras (*feature_names*, aquí las palabras son rasgos distintivos de la matriz) para construir la matriz TF-IDF y se verifica que el vocabulario sea el mismo que el creado en la práctica. Posteriormente, se calcula automáticamente la métrica TF-IDF apliacada a los cinco textos vectorizados: *tf_idf = vectorizer.fit_transform(textos)* y se muestra el resultado en forma de matriz dispersa, pues solo muestra los valores distintos a cero.
En esa matriz la primera columna representa el número del documento, la segunda el número de la palabra en el documento y la tercera, el correspondiente valor TF-IDF.**

Con la siguiente función pasamos una matriz a una lista de diccionarios:

In [ ]:
def sparseMatrixToDics (matrix, n, m, feature_names):
  res = [0] * n
  for i in range(0,n): # es igual que range(n)
    res[i]=dict()
    for j in range(0,m):
      valaux = matrix[i,j]
      if valaux != 0.0:
        res[i][feature_names[j]] = valaux
  return res

**Ahora para poder comparar el resultado con el nuestro, representado en forma de dicionario, se transforma la matriz a una lista de diccionarios mediante una función que tiene cuatro argumentos: la matriz TF-IDF en formato disperso, en la que cada celda contiene los pesos de cada palabra; n, el número de filas, que es el número de documentos de la matriz; m, el número de columnas, que es el número de palabras del vocabulario, y *feature_names* que son las palabras en sí a las que se asignarán los valores TF-IDF.**







**La función crea una lista, *res* con n elementos, con un valor inicial de cero. Luego, mediante un bucle, se recorre cada documento de la matriz y se reemplaza cada elemento de la lista con un diccionario vacío.
A continuación, se itera sobre cada palabra del vocabulario para extraer el valor TF-IDF correspondiente de la matriz.
Si el valor es distinto de cero, se almacena en el diccionario del documento, y se la asigna como clave la palabra correspondiente de feature_names y como valor el peso TF-IDF.
Finalmente, la función devuelve la lista *res*, donde cada documento se representa como un diccionario.**

In [ ]:
sparseMatrixToDics (tf_idf, len(textos), len(vocabulario), feature_names)

[{'la': 0.19654575253083423,
  'en': 0.5524763946578989,
  'de': 0.6971408403404858,
  'alquiler': 0.41247333154673},
 {'la': 0.3375338265523302,
  'en': 0.4743920160255332,
  'de': 0.39907351927997176,
  'calentamiento': 0.7083526362438907},
 {'la': 0.27765951098422603,
  'en': 0.3902407546227053,
  'de': 0.6565656505710366,
  'mundo': 0.5826996618170748},
 {'la': 0.7797586601687984,
  'de': 0.30730849100478064,
  'salud': 0.5454703688085403},
 {'la': 0.430165282498796, 'no': 0.9027501480103624}]

**Se puede comprobar que los resultados del cáculo manual y del automático son los mismos, por lo que parece que el ejercicio ha sido correctamente realizado. Además, se puede concluir que la representación en forma de listas de diccionarios facilita la legibilidad e interpretación de los valores TF-IDF.**

**Ejercicio opcional 4 (abierto)**. Comparar los valores obtenidos por el modelo tf-idf sklearn anterior con el que habéis programado antes. Si los valores no coinciden, intentad re-programar vuestro modelo empleando las formulas de sklearn en:

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#

Lo consigáis o no, explicad a qué se pueden deber las discrepancias.

La librería de sklearn también permite vectorizar con bows. Aquí sí que se comprueba que nuestros cálculos coinciden con los de la librería.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cvectorizer = CountVectorizer(vocabulary=vocabulario)

In [ ]:
skbows = cvectorizer.fit_transform(textos)

In [ ]:
print(skbows)

In [ ]:
sparseMatrixToDics (skbows, len(textos), len(vocabulario), feature_names)

In [ ]:
bows

Terminamos esta sesión doble de prácticas con dos ejercicios opcionales.

**Ejercicio opcional 5**. Obtener el modelo tf-idf de una colección de textos extraídas de algún contexto real. Explicad qué vocabulario habéis elegido. Comparar con los resultados de sklearn. ¿Convendría cambiar la definición de "coincide"?

**Ejercicio opcional 6 (teórico)**. El algún momento de los programas que hemos ido escribiendo hemos perdido la flexibilidad que nos ofrece "coincide" (es decir, hemos comparado las cadenas con ==, la igualdad literal, y hemos perdido la posibilidad de comparar tras *stemming* o *lematización*, etc.). Detectad en qué funciones ha sido y proponed qué soluciones se os ocurren para poder recuperar esa flexibilidad.